In [51]:
!pip install pypdf
!pip install joblib
!pip install pandas
!pip install scikit-learn
!pip install xlrd
!pip install youtube_transcript_api
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-pac

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd


# 2025 Canadian Election Data
videos = pd.read_excel('../data/2025/political-classification-model-videos.xlsx')
video_links = videos['url'].tolist()
transcript_fetcher = YouTubeTranscriptApi()
parties = []
party_classes = []
texts = []

party_ids = {'conservative': 0, 'liberal': 1, 'ndp': 2, 'green': 3}
party_names = ['conservative', 'liberal', 'ndp', 'green']

for index, row in videos.iterrows():
    video_id = row['url'].split("=")[1]
    
    video_transcript = transcript_fetcher.fetch(video_id)
    
    for snippet in video_transcript:
        text = snippet.text
        
        # Restricting text to a specific length
        if '[' not in text and len(text) > 15:            
            texts.append(text)
            party_classes.append(party_ids[row['party']])
            parties.append(row['party'])
            
data = pd.DataFrame({'text': texts, 'label': party_classes, 'label_text': parties})

In [4]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")

# Split dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["text"].tolist(), data["label"].tolist(), test_size=0.2, random_state=42
)

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels}).map(tokenize_function, batched=True)
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels}).map(tokenize_function, batched=True)

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=4)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    auto_find_batch_size=True,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
)

# Train the model
trainer.train()

# Save model
model.save_pretrained("./canadian_political_bert")
tokenizer.save_pretrained("./canadian_political_bert")

print("Model training complete and saved.")


Map: 100%|██████████| 4502/4502 [00:00<00:00, 9305.23 examples/s] 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/isss11/.local/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_1169/1195854238.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.323800,1.324084
2,1.336900,1.303068
3,1.322800,1.303723
4,1.333500,1.306389
5,1.304600,1.303037


Model training complete and saved.


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def predict_political_affiliation(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move to GPU if available

    with torch.no_grad():
        print(inputs)
        
        outputs = model(**inputs)
    
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # Map class index to party
    return party_names[predicted_class]

print(predict_political_affiliation("I think the wealthiest need to pay their fair share of taxes. We need to raise taxes. We also need to deal with climate change."))


{'input_ids': tensor([[  101,  1045,  2228,  1996, 27809,  2342,  2000,  3477,  2037,  4189,
          3745,  1997,  7773,  1012,  2057,  2342,  2000,  5333,  7773,  1012,
          2057,  2036,  2342,  2000,  3066,  2007,  4785,  2689,  1012,   102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]], device='cuda:0')}
conservative


In [ ]:
# Test Predictions on Training Dataset (to fix later)